If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [1]:
## If running in Colab
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# v8+ should be fine
!java -version

openjdk version "11.0.22" 2024-01-16
OpenJDK Runtime Environment (build 11.0.22+7-post-Ubuntu-0ubuntu222.04.1)
OpenJDK 64-Bit Server VM (build 11.0.22+7-post-Ubuntu-0ubuntu222.04.1, mixed mode, sharing)


In [24]:
import os
from subprocess import Popen, PIPE, STDOUT
import pandas as pd

splitter_jar_path = '../../SentenceSplitter.jar'
splitter_model_path = '../../es-sentence-splitter-model-spaccc.bin'
text_path = '../../multicardioner_train+dev+test+bg_240502/test+background/es' #'../../multicardioner_train+dev+test+bg_240502/track1/distemist_train/brat'
annotations_path = '' #'../../multicardioner_train+dev+test+bg_240502/track1/distemist_train/brat'
output_path = '../../multicardioner_train+dev+test+bg_240502/test+background/es/sentences' #'../../multicardioner_train+dev+test+bg_240502/track1/distemist_train/sentences' 
sents_per_batch = 1 # max number of sentences in each output file
is_test = True

In [ ]:
from pathlib import Path

sentences = []
for file_name in os.listdir(text_path):
    if '.txt' in file_name: # and 'casos_clinicos_cardiologia129' in file_name:
        file_name_no_ext = file_name.rstrip('.txt')
                 
        file_path = Path(f'{text_path}/{file_name}')
        full_text = file_path.read_text()
        
        p = Popen(['java', '-jar', splitter_jar_path , f'{text_path}/{file_name}', splitter_model_path], stdout=PIPE, stderr=STDOUT, text=True)

        print("Processing: ", file_name_no_ext)
        
        #T105	FARMACO 911 922	azatioprina
        if not is_test:
            annotations = pd.read_csv(f'{annotations_path}/{file_name_no_ext}.ann', sep='\t', 
                                      names=['ann_type', 'entity_type', 'text'], encoding='utf-8', keep_default_na=False)

            # drop annotator comments
            annotations.drop(annotations[annotations['ann_type'].str.startswith('#')].index, inplace=True)

            annotations['start_pos'] = annotations['entity_type'].apply(lambda v: int(v.split()[1]))
            annotations['end_pos'] = annotations['entity_type'].apply(lambda v: int(v.split()[2]))
            annotations['entity_type'] = annotations['entity_type'].apply(lambda v: v.split()[0])
            annotations.sort_values(by=['start_pos'], inplace=True, ignore_index=True)
        else:
            annotations = pd.DataFrame(columns=['ann_type', 'text'])
        
        total_chars = 0
        annotation_index = 0
        total_annotations = len(annotations)
        line_number = 0
        batch = []
        batch_number = 0
        for line in p.stdout:
            line_number += 1
            batch.append(line)

            if line_number % sents_per_batch == 0:
                batch_number += 1
                current_batch_annotation_indecies = []
                text = ''.join(batch)
                #max_index = total_chars + len(text) # - 1;
                #max_index = total_chars + full_text[total_chars:].find(text) + len(text) - 1
                batch_start = total_chars + full_text[total_chars:].find(text.strip()) #+ len(text) - 1
                batch_end = batch_start + len(text)
                #print(text, batch_start, batch_end)
                if batch_start < 0:
                    print('batch_start', full_text[total_chars:], text, total_chars)
                
                while annotation_index < total_annotations and annotations.iloc[annotation_index]['start_pos'] >= batch_start and annotations.iloc[annotation_index]['start_pos'] < batch_end:
                    annotation_text = str(annotations.iloc[annotation_index]['text'])
                    #print(annotation_text)
                    #new_start_pos = text.find(annotation_text)
                    new_start_pos = annotations.iloc[annotation_index]['start_pos'] - batch_start
                    
                    if new_start_pos < 0:
                        print('new_start_pos', text, annotation_text, batch_number, max_index)
                        
                    annotations.at[annotation_index, 'start_pos'] = new_start_pos
                    annotations.at[annotation_index, 'end_pos'] = new_start_pos + len(annotation_text)
                    current_batch_annotation_indecies.append(annotation_index)
                    annotation_index += 1

                format_batch_number = str(batch_number).zfill(3)
                with open(f'{output_path}/splits_{sents_per_batch}/{file_name_no_ext}-b-{format_batch_number}-{batch_start}.txt', 'w', encoding='utf-8') as out_file:
                    out_file.write(text)

                # file_name_no_ext, batch_number, batch_start, text
                sentences.append({
                    'filename': file_name_no_ext,
                    'batch_number': batch_number,
                    'batch_start': batch_start,
                    'text': text
                })
                    
                current_batch_annotations = annotations.iloc[current_batch_annotation_indecies,].copy().reset_index(drop=True)
                current_batch_annotations['entity_type_with_positions'] = None
                if not current_batch_annotations.empty:
                    current_batch_annotations['row_name'] = current_batch_annotations.apply(lambda row: row.name + 1, axis=1)
                    current_batch_annotations['ann_type'] = current_batch_annotations.apply(lambda row: f'{row["ann_type"].rstrip("0123456789")}{row.name + 1}', axis=1)
                    current_batch_annotations['entity_type_with_positions'] = current_batch_annotations.apply(lambda row: f'{row["entity_type"]} {row["start_pos"]} {row["end_pos"]}', axis=1)
                    current_batch_annotations.drop(columns=['start_pos', 'end_pos', 'entity_type'], inplace=True)

                current_batch_annotations.to_csv(f'{output_path}/splits_{sents_per_batch}/{file_name_no_ext}-b-{format_batch_number}-{batch_start}.ann', sep='\t', header=False, index=False, columns=['ann_type', 'entity_type_with_positions', 'text'], encoding='utf-8')

                total_chars = batch_end #- 1 #len(text) # -1
                batch.clear()
                
        if annotation_index < total_annotations:
            print('missing annotations')
        #break

Processing:  multicardioner_test+bg_1
Processing:  multicardioner_test+bg_10
Processing:  multicardioner_test+bg_100
Processing:  multicardioner_test+bg_1000
Processing:  multicardioner_test+bg_1001
Processing:  multicardioner_test+bg_1002
Processing:  multicardioner_test+bg_1003
Processing:  multicardioner_test+bg_1004
Processing:  multicardioner_test+bg_1005
Processing:  multicardioner_test+bg_1006
Processing:  multicardioner_test+bg_1007
Processing:  multicardioner_test+bg_1008
Processing:  multicardioner_test+bg_1009
Processing:  multicardioner_test+bg_101
Processing:  multicardioner_test+bg_1010
Processing:  multicardioner_test+bg_1011
Processing:  multicardioner_test+bg_1012
Processing:  multicardioner_test+bg_1013
Processing:  multicardioner_test+bg_1014
Processing:  multicardioner_test+bg_1015
Processing:  multicardioner_test+bg_1016
Processing:  multicardioner_test+bg_1017
Processing:  multicardioner_test+bg_1018
Processing:  multicardioner_test+bg_1019
Processing:  multicardi

Processing:  multicardioner_test+bg_118
Processing:  multicardioner_test+bg_1180
Processing:  multicardioner_test+bg_1181
Processing:  multicardioner_test+bg_1182
Processing:  multicardioner_test+bg_1183
Processing:  multicardioner_test+bg_1184
Processing:  multicardioner_test+bg_1185
Processing:  multicardioner_test+bg_1186
Processing:  multicardioner_test+bg_1187
Processing:  multicardioner_test+bg_1188
Processing:  multicardioner_test+bg_1189
Processing:  multicardioner_test+bg_119
Processing:  multicardioner_test+bg_1190
Processing:  multicardioner_test+bg_1191
Processing:  multicardioner_test+bg_1192
Processing:  multicardioner_test+bg_1193
Processing:  multicardioner_test+bg_1194
Processing:  multicardioner_test+bg_1195
Processing:  multicardioner_test+bg_1196
Processing:  multicardioner_test+bg_1197
Processing:  multicardioner_test+bg_1198
Processing:  multicardioner_test+bg_1199
Processing:  multicardioner_test+bg_12
Processing:  multicardioner_test+bg_120
Processing:  multicar

Processing:  multicardioner_test+bg_1361
Processing:  multicardioner_test+bg_1362
Processing:  multicardioner_test+bg_1363
Processing:  multicardioner_test+bg_1364
Processing:  multicardioner_test+bg_1365
Processing:  multicardioner_test+bg_1366
Processing:  multicardioner_test+bg_1367
Processing:  multicardioner_test+bg_1368
Processing:  multicardioner_test+bg_1369
Processing:  multicardioner_test+bg_137
Processing:  multicardioner_test+bg_1370
Processing:  multicardioner_test+bg_1371
Processing:  multicardioner_test+bg_1372
Processing:  multicardioner_test+bg_1373
Processing:  multicardioner_test+bg_1374
Processing:  multicardioner_test+bg_1375
Processing:  multicardioner_test+bg_1376
Processing:  multicardioner_test+bg_1377
Processing:  multicardioner_test+bg_1378
Processing:  multicardioner_test+bg_1379
Processing:  multicardioner_test+bg_138
Processing:  multicardioner_test+bg_1380
Processing:  multicardioner_test+bg_1381
Processing:  multicardioner_test+bg_1382
Processing:  multi

Processing:  multicardioner_test+bg_1542
Processing:  multicardioner_test+bg_1543
Processing:  multicardioner_test+bg_1544
Processing:  multicardioner_test+bg_1545
Processing:  multicardioner_test+bg_1546
Processing:  multicardioner_test+bg_1547
Processing:  multicardioner_test+bg_1548
Processing:  multicardioner_test+bg_1549
Processing:  multicardioner_test+bg_155
Processing:  multicardioner_test+bg_1550
Processing:  multicardioner_test+bg_1551
Processing:  multicardioner_test+bg_1552
Processing:  multicardioner_test+bg_1553
Processing:  multicardioner_test+bg_1554
Processing:  multicardioner_test+bg_1555
Processing:  multicardioner_test+bg_1556
Processing:  multicardioner_test+bg_1557
Processing:  multicardioner_test+bg_1558
Processing:  multicardioner_test+bg_1559
Processing:  multicardioner_test+bg_156
Processing:  multicardioner_test+bg_1560
Processing:  multicardioner_test+bg_1561
Processing:  multicardioner_test+bg_1562
Processing:  multicardioner_test+bg_1563
Processing:  multi

Processing:  multicardioner_test+bg_1723
Processing:  multicardioner_test+bg_1724
Processing:  multicardioner_test+bg_1725
Processing:  multicardioner_test+bg_1726
Processing:  multicardioner_test+bg_1727
Processing:  multicardioner_test+bg_1728
Processing:  multicardioner_test+bg_1729
Processing:  multicardioner_test+bg_173
Processing:  multicardioner_test+bg_1730
Processing:  multicardioner_test+bg_1731
Processing:  multicardioner_test+bg_1732
Processing:  multicardioner_test+bg_1733
Processing:  multicardioner_test+bg_1734
Processing:  multicardioner_test+bg_1735
Processing:  multicardioner_test+bg_1736
Processing:  multicardioner_test+bg_1737
Processing:  multicardioner_test+bg_1738
Processing:  multicardioner_test+bg_1739
Processing:  multicardioner_test+bg_174
Processing:  multicardioner_test+bg_1740
Processing:  multicardioner_test+bg_1741
Processing:  multicardioner_test+bg_1742
Processing:  multicardioner_test+bg_1743
Processing:  multicardioner_test+bg_1744
Processing:  multi

...

Processing:  multicardioner_test+bg_2087
Processing:  multicardioner_test+bg_2088
Processing:  multicardioner_test+bg_2089
Processing:  multicardioner_test+bg_209
Processing:  multicardioner_test+bg_2090
Processing:  multicardioner_test+bg_2091
Processing:  multicardioner_test+bg_2092
Processing:  multicardioner_test+bg_2093
Processing:  multicardioner_test+bg_2094
Processing:  multicardioner_test+bg_2095
Processing:  multicardioner_test+bg_2096
Processing:  multicardioner_test+bg_2097
Processing:  multicardioner_test+bg_2098
Processing:  multicardioner_test+bg_2099
Processing:  multicardioner_test+bg_21
Processing:  multicardioner_test+bg_210
Processing:  multicardioner_test+bg_2100
Processing:  multicardioner_test+bg_2101
Processing:  multicardioner_test+bg_2102
Processing:  multicardioner_test+bg_2103
Processing:  multicardioner_test+bg_2104
Processing:  multicardioner_test+bg_2105
Processing:  multicardioner_test+bg_2106
Processing:  multicardioner_test+bg_2107
Processing:  multica

Processing:  multicardioner_test+bg_2268
Processing:  multicardioner_test+bg_2269
Processing:  multicardioner_test+bg_227
Processing:  multicardioner_test+bg_2270
Processing:  multicardioner_test+bg_2271
Processing:  multicardioner_test+bg_2272
Processing:  multicardioner_test+bg_2273
Processing:  multicardioner_test+bg_2274
Processing:  multicardioner_test+bg_2275
Processing:  multicardioner_test+bg_2276
Processing:  multicardioner_test+bg_2277
Processing:  multicardioner_test+bg_2278
Processing:  multicardioner_test+bg_2279
Processing:  multicardioner_test+bg_228
Processing:  multicardioner_test+bg_2280
Processing:  multicardioner_test+bg_2281
Processing:  multicardioner_test+bg_2282
Processing:  multicardioner_test+bg_2283
Processing:  multicardioner_test+bg_2284
Processing:  multicardioner_test+bg_2285
Processing:  multicardioner_test+bg_2286
Processing:  multicardioner_test+bg_2287
Processing:  multicardioner_test+bg_2288
Processing:  multicardioner_test+bg_2289
Processing:  multi

Processing:  multicardioner_test+bg_2449
Processing:  multicardioner_test+bg_245
Processing:  multicardioner_test+bg_2450
Processing:  multicardioner_test+bg_2451
Processing:  multicardioner_test+bg_2452
Processing:  multicardioner_test+bg_2453
Processing:  multicardioner_test+bg_2454
Processing:  multicardioner_test+bg_2455
Processing:  multicardioner_test+bg_2456
Processing:  multicardioner_test+bg_2457
Processing:  multicardioner_test+bg_2458
Processing:  multicardioner_test+bg_2459
Processing:  multicardioner_test+bg_246
Processing:  multicardioner_test+bg_2460
Processing:  multicardioner_test+bg_2461
Processing:  multicardioner_test+bg_2462
Processing:  multicardioner_test+bg_2463
Processing:  multicardioner_test+bg_2464
Processing:  multicardioner_test+bg_2465
Processing:  multicardioner_test+bg_2466
Processing:  multicardioner_test+bg_2467
Processing:  multicardioner_test+bg_2468
Processing:  multicardioner_test+bg_2469
Processing:  multicardioner_test+bg_247
Processing:  multic

Processing:  multicardioner_test+bg_263
Processing:  multicardioner_test+bg_2630
Processing:  multicardioner_test+bg_2631
Processing:  multicardioner_test+bg_2632
Processing:  multicardioner_test+bg_2633
Processing:  multicardioner_test+bg_2634
Processing:  multicardioner_test+bg_2635
Processing:  multicardioner_test+bg_2636
Processing:  multicardioner_test+bg_2637
Processing:  multicardioner_test+bg_2638
Processing:  multicardioner_test+bg_2639
Processing:  multicardioner_test+bg_264
Processing:  multicardioner_test+bg_2640
Processing:  multicardioner_test+bg_2641
Processing:  multicardioner_test+bg_2642
Processing:  multicardioner_test+bg_2643
Processing:  multicardioner_test+bg_2644
Processing:  multicardioner_test+bg_2645
Processing:  multicardioner_test+bg_2646
Processing:  multicardioner_test+bg_2647
Processing:  multicardioner_test+bg_2648
Processing:  multicardioner_test+bg_2649
Processing:  multicardioner_test+bg_265
Processing:  multicardioner_test+bg_2650
Processing:  multic

Processing:  multicardioner_test+bg_2810
Processing:  multicardioner_test+bg_2811
Processing:  multicardioner_test+bg_2812
Processing:  multicardioner_test+bg_2813
Processing:  multicardioner_test+bg_2814
Processing:  multicardioner_test+bg_2815
Processing:  multicardioner_test+bg_2816


In [ ]:
df_sentences = pd.DataFrame.from_records(sentences)
df_sentences.head()

In [ ]:
df_sentences.to_csv(f'{output_path}/sentences.tsv', sep='\t', index=False)